Import libraries

In [ ]:
import pandas as pd
import numpy as np
import re
import json
import gensim
import math
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import KeyedVectors
import h5py

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Membuka data

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel("data_sastra_inggris_full.xlsx")

In [ ]:
df.head(5)

,Nomor,Kalimat
0,1,"I brothers, thee of thee, because of thee, bec..."
1,2,"Becoming increasingly, because rinubed in the ..."
2,3,"Thus, I do not want, I think this is why the c..."
3,4,Determine your question:
4,5,"How it's sitting on me, and the seakindness."


pop data nomor karena tidak dibutuhkan

In [ ]:
df.pop('Nomor')

,Nomor
0,1
1,2
2,3
3,4
4,5
...,...
1246,1247
1247,1248
1248,1249
1249,1250


In [ ]:
df.head(5)

,Kalimat
0,"I brothers, thee of thee, because of thee, bec..."
1,"Becoming increasingly, because rinubed in the ..."
2,"Thus, I do not want, I think this is why the c..."
3,Determine your question:
4,"How it's sitting on me, and the seakindness."


In [ ]:
len(df)

1251

In [ ]:
pip install pandas openpyxl

Data preprocessing

hanya ada data alphabets + lower

In [ ]:
def text_clean(corpus):
    cleaned_corpus = []
    for row in corpus:
        qs = []
        for word in row.split():
            p1 = re.sub(pattern='[^a-zA-Z]',repl=' ',string=word)
            p1 = p1.lower()
            qs.append(p1)
        cleaned_corpus.append(' '.join(qs))
    return pd.Series(cleaned_corpus)

Stopwords_removal

1. Membuat fungsi stopword_removal untuk menghilangkan kata umum yang tidak memiliki bobot makna pada kalimat keseluruhan, menggunakan perulangan
2. kata tanya dikecualikan agar tidak dihilangkan menggunakan variabel wh_words dilakukan stop.remove()

In [ ]:
def stopwords_removal(corpus):
    wh_words = ['who', 'what', 'when', 'why', 'how', 'which', 'where', 'whom']
    stop = set(stopwords.words('english'))
    for word in wh_words:
        stop.remove(word)
    corpus = [[x for x in x.split() if x not in stop] for x in corpus]
    return corpus

Lemmatization

Membuat fungsi lemmatize untuk melakukan lemmatization setiap katanya menggunakan dasar kamus WordNetLemmatizer()

In [ ]:
def lemmatize(corpus):
    lem = WordNetLemmatizer()
    corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in corpus]
    return corpus

Stemming

Membuat fungsi stem untuk melakukan stemming.
1. jika stemming menggunakan snowball dari parameternya, maka akan memanggil fungsi SnowballStemmer
2. selain parameter snowball, maka akan menggunakan fungsi PorterStemmer

In [ ]:
def stem(corpus, stem_type = None):
    if stem_type == 'snowball':
        stemmer = SnowballStemmer(language = 'english')
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    else :
        stemmer = PorterStemmer()
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    return corpus

Membuat fungsi preprocess yang akan melakukan semua proses preprocessing dengan melemparkannya pada fungsi lain sesuai kebutuhannya

In [ ]:
def preprocess(corpus, cleaning = True, stemming = False, stem_type = None, lemmatization = False, remove_stopwords = True):

    if cleaning == True:
        corpus = text_clean(corpus)

    if remove_stopwords == True:
        corpus = stopwords_removal(corpus)
    else :
        corpus = [[x for x in x.split()] for x in corpus]

    if lemmatization == True:
        corpus = lemmatize(corpus)


    if stemming == True:
        corpus = stem(corpus, stem_type)

    corpus = [' '.join(x) for x in corpus]

    return corpus

membuat variabel untuk dijadikan sebagai pengecualian pada saat text_clean (yang hanya memperbolehkan alphabet dan numerik)

In [ ]:
df['Kalimat']

,Kalimat
0,"I brothers, thee of thee, because of thee, bec..."
1,"Becoming increasingly, because rinubed in the ..."
2,"Thus, I do not want, I think this is why the c..."
3,Determine your question:
4,"How it's sitting on me, and the seakindness."
...,...
1246,27. The power of the site |Party of the Parani...
1247,28. Tan vague the surminister |Ana Sasmita is ...
1248,29. Here you have focused |The Spirit |Aywa Um...
1249,30. And again Aywa Aywa |Mering people 'If Mid...


In [ ]:
headlines = preprocess(df['Kalimat'], lemmatization = True, remove_stopwords = True)

In [ ]:
pip install googletrans==4.0.0-rc1

In [ ]:
from googletrans import Translator
import time

translator = Translator()
language = 'jw'

headlines_jw = []

sleep_time = 3

for i in range(len(headlines)):
    try:
        if headlines[i]:  # cek apakah elemen tidak None atau kosong
            translation = translator.translate(headlines[i], dest=language)
            if translation and translation.text:  # cek apakah hasil terjemahan valid
                headlines_jw.append(translation.text.lower())
            else:
                continue
        else:
            continue
    except Exception as e:
        print(f"Error in translating {headlines[i]}: {e}")
    time.sleep(sleep_time)

print(len(headlines_jw))

1015


In [ ]:
headlines_jw[0:5]

['sedulur sampeyan sampeyan nggawe jagad kamulyan kepiye bisa mbayar maneh yen bisa mulihake salawas-lawase',
 'dadi tambah akeh dampor massa wong manca anasabi surya sing ora bisa',
 'saengga pengin mikir kenapa manungsa nggawe seneng kaya panglipur panglipur manungsa martusit uga nggawe inflamasi',
 'nemtokake pitakon',
 'cara lenggah']

melakukan preprocessing data agar menjadi matrix

In [ ]:
X = headlines_jw

In [ ]:
splitX = []
for i in range(len(X)):
    if X[i] and X[i].split():
            splitX.append(X[i].split())
    else:
        continue
print(splitX)

[['sedulur', 'sampeyan', 'sampeyan', 'nggawe', 'jagad', 'kamulyan', 'kepiye', 'bisa', 'mbayar', 'maneh', 'yen', 'bisa', 'mulihake', 'salawas-lawase'], ['dadi', 'tambah', 'akeh', 'dampor', 'massa', 'wong', 'manca', 'anasabi', 'surya', 'sing', 'ora', 'bisa'], ['saengga', 'pengin', 'mikir', 'kenapa', 'manungsa', 'nggawe', 'seneng', 'kaya', 'panglipur', 'panglipur', 'manungsa', 'martusit', 'uga', 'nggawe', 'inflamasi'], ['nemtokake', 'pitakon'], ['cara', 'lenggah'], ['wrohutanira', 'isih', 'kepenak'], ['kesadharan', 'kasus', 'kasusahan', 'kanggo', 'ngarsane', 'fury', 'ngupaya'], ['kandha'], ['mikir', 'kaya', 'katrr', 'potos', 'cara', 'ngirit', 'ngluwihi', 'mata'], ['suwe', 'banget', 'nggawe', 'perabot', 'nerangake', 'kahanan', 'sing', 'ditemokake'], ['fations', 'fistik', 'nate', 'inggris', 'tengah'], ['ngerteni', 'kasunyatan', 'bebener', 'urip', 'tanpa', 'reader', 'alam', 'west', 'wor', 'ngerti'], ['minangka', 'prihatin', 'babagan', 'kiilaka', 'sing', 'cilama.', 'chilicalne', 'minangka', '

In [ ]:
kata_jw = []
for i in range(len(splitX)):
  for j in range(len(splitX[i])):
    kata_jw.append(splitX[i][j])

print(kata_jw)

['sedulur', 'sampeyan', 'sampeyan', 'nggawe', 'jagad', 'kamulyan', 'kepiye', 'bisa', 'mbayar', 'maneh', 'yen', 'bisa', 'mulihake', 'salawas-lawase', 'dadi', 'tambah', 'akeh', 'dampor', 'massa', 'wong', 'manca', 'anasabi', 'surya', 'sing', 'ora', 'bisa', 'saengga', 'pengin', 'mikir', 'kenapa', 'manungsa', 'nggawe', 'seneng', 'kaya', 'panglipur', 'panglipur', 'manungsa', 'martusit', 'uga', 'nggawe', 'inflamasi', 'nemtokake', 'pitakon', 'cara', 'lenggah', 'wrohutanira', 'isih', 'kepenak', 'kesadharan', 'kasus', 'kasusahan', 'kanggo', 'ngarsane', 'fury', 'ngupaya', 'kandha', 'mikir', 'kaya', 'katrr', 'potos', 'cara', 'ngirit', 'ngluwihi', 'mata', 'suwe', 'banget', 'nggawe', 'perabot', 'nerangake', 'kahanan', 'sing', 'ditemokake', 'fations', 'fistik', 'nate', 'inggris', 'tengah', 'ngerteni', 'kasunyatan', 'bebener', 'urip', 'tanpa', 'reader', 'alam', 'west', 'wor', 'ngerti', 'minangka', 'prihatin', 'babagan', 'kiilaka', 'sing', 'cilama.', 'chilicalne', 'minangka', 'kewarganea', 'sing', 'cip

In [ ]:
kata_jw = set(kata_jw)
fix_kata_jw = list(kata_jw)

download file

In [ ]:
array_nomor = []
for i in range(1,len(fix_kata_jw)+1):
    array_nomor.append(i)

In [ ]:
# Menggabungkan array menjadi DataFrame
df = pd.DataFrame({
    'Nomor': array_nomor,
    'Kalimat': fix_kata_jw
})

# Menyimpan DataFrame ke file Excel
df.to_excel('data_preprocessing_sastra_jawa_full.xlsx', index=False, engine='openpyxl')

print("Data telah disimpan ke file Excel.")

Data telah disimpan ke file Excel.
